# Export der Gemeinden, Bezirke und Bundesländer

Liest die Gemeindekennzahlen, Bezirke und Bundesländer aus den Shapefiles der Verwaltungsgrenzen.

Quellen:
- **BEV: Verwaltungsgrenzen - VGD**: https://www.bev.gv.at/Services/Downloads/Produktbezogene-Downloads/Unentgeltliche-Produkte/Kataster-Verzeichnisse.html
- **Gliederung Österreichs in Politische Bezirke:** https://www.data.gv.at/katalog/dataset/stat_gliederung-osterreichs-in-politische-bezirke131e2
- **Wahlkreiseinteilung:** https://www.bmi.gv.at/412/Nationalratswahlen/Wahlkreiseinteilung.aspx

In [ ]:
import pandas as pd, datetime as dt, numpy as np
import geopandas as gp  # https://geopandas.org/en/stable/docs/user_guide.html
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 128)

In [ ]:
bev_shapefile = "VGD.shp"    # Verwaltungsgrenzen aus dem BEV Katalog
bezirke_shapefile = "STATISTIK_AUSTRIA_POLBEZ_20230101.shp"    # Bezirksgrenzen von Statistik Austria (für die Wiener Bezirke)
# Aus dem prj File der Verwaltungsgrenzen.
crs = 'PROJCS["MGI_Austria_Lambert",GEOGCS["GCS_MGI",DATUM["D_MGI",SPHEROID["Bessel_1841",6377397.155,299.1528128]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",400000.0],PARAMETER["Central_Meridian",13.33333333333333],PARAMETER["Standard_Parallel_1",46.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",47.5],UNIT["Meter",1.0]]'

In [ ]:
# Gemeinden ohne Wien lesen. Wien hat Gemeindebezirke, diese sind aber hier nicht enthalten.
katastralgemeinden = gp.read_file(bev_shapefile, encoding="utf-8") \
    [['KG_NR', 'GKZ', 'BKZ', 'BL_KZ', 'KG', 'PG', 'PB', 'BL', 'geometry']] \
    .rename({'BL_KZ': 'BLKZ'}, axis=1) \
    .astype({"KG_NR": int, "GKZ": int, "BKZ": int, "BLKZ": int, "KG": "string", "PG": "string", "PB": "string", "BL": "string"}) \
    .query("GKZ < 90000") \
    .set_crs(crs, allow_override=True)
katastralgemeinden["PB"] = katastralgemeinden.PB.str.replace("(", " (", regex=False)
# Gemeindebezirke von Wien lesen. Wir bilden diese Bezirke als eigene Gemeinde ab.
bezirke_wien = gp.read_file(bezirke_shapefile, encoding="utf-8", crs="EPSG:31287") \
    .rename({'g_id': "BKZ", "g_name": "PG"}, axis=1) \
    .astype({"BKZ": int, "PG": "string"}) \
    .query("BKZ >= 901") \
    .set_crs(crs, allow_override=True)
bezirke_wien["GKZ"] = bezirke_wien.BKZ * 100 + 1
bezirke_wien["PG"] = bezirke_wien.PG.str.replace(",", ", ", regex=False)
bezirke_wien["PG"] = bezirke_wien.PG.str.replace("\s+", " ", regex=True)
bezirke_wien["PB"] = bezirke_wien.PG
bezirke_wien["BLKZ"] = 9
bezirke_wien["BL"] = "Wien"
bezirke_wien["BL"] = bezirke_wien.BL.astype("string")

In [ ]:
wahlkreise = pd.read_csv("wahlkreise.tsv", encoding="utf-8", sep="\t").convert_dtypes().set_index("BKZ")
gemeinden = katastralgemeinden.dissolve(by="GKZ").drop(["KG_NR", "KG"], axis=1).reset_index()
gemeinden = pd.concat([gemeinden, bezirke_wien]).join(wahlkreise, on="BKZ")
bezirke = gemeinden.dissolve(by="BKZ").drop(["GKZ", "PG"], axis=1).reset_index()
bundeslaender = gemeinden.dissolve(by="BLKZ").drop(["GKZ", "PG", "BKZ", "PB", "WKNR", "WKNAME"], axis=1).reset_index()

Den Mittelpunkt der Objekte berechnen, nach WSG84 konvertieren und die Geoinformation löschen.

In [ ]:
for df in [gemeinden, bezirke, bundeslaender]:
    df["CENTER"] = df.centroid
    df["AREA"] = round(df.area / 1_000_000, 3)
    df.set_geometry("CENTER", inplace=True)
    df.to_crs("WGS84", inplace=True)
    df["CENTER_X"] = round(df.CENTER.x, 6)
    df["CENTER_Y"] = round(df.CENTER.y, 6)
    df.drop(["CENTER", "geometry"], axis=1, inplace=True)


In [ ]:
gemeinden.sort_values("GKZ").to_csv("gemeinden.tsv", sep="\t", encoding="utf-8", index=False)
bezirke.sort_values("BKZ").to_csv("bezirke.tsv", sep="\t", encoding="utf-8", index=False)
bundeslaender.sort_values("BLKZ").to_csv("bundeslaender.tsv", sep="\t", encoding="utf-8", index=False)